In [1]:
import numpy as np 
import pandas as pd 

import math
import random 
import os 
import cv2
import timm

from tqdm import tqdm 

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import torch 
from torch.utils.data import Dataset 
from torch import nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
import plotly.express as px

import gc
#import cudf
#import cuml
#import cupy
#from cuml.neighbors import NearestNeighbors

In [2]:
class Config:
    META_CSV = '/mnt/hdd1/wearly/deep_rec/separ_meta.csv'
    
    DATA_DIR = '/mnt/hdd1/wearly/compatibility_rec/data/images'
    TRAIN_CSV = '/mnt/hdd1/wearly/deep_rec/separ_train.csv'
    #TEST_CSV = '/mnt/hdd1/wearly/deep_rec/data/test_csv/150limit_test.csv'
    SEED = 225

    IMG_SIZE = 224
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

    BATCH_SIZE = 64
    N_FOLDS = 10
    FC_DIM = 512
    
    NUM_WORKERS = 4
    DEVICE = 'cuda:0'
     
    CLASSES = 5062
    SCALE = 30 
    MARGIN = 0.5

    MODEL_NAME = 'tf_efficientnet_b3'
    MODEL_PATH = './separ_tf_efficientnet_b3_50_Weights/tf_efficientnet_b3_11EpochStep_adam.pt'
    


#### Utility function

In [3]:
def read_dataset():
    df = pd.read_csv(Config.TRAIN_CSV,index_col=0)
    #df = df.head() #test
    #df_cu = cudf.DataFrame(df)
    image_paths = Config.DATA_DIR + '/' + df['image_name']
    return df,image_paths

def seed_setting(seed=Config.SEED):
    random.seed(seed)
    torch.cuda.set_device(Config.DEVICE)
#     set_seed(seed, reproducible=True)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True
    
seed_setting(Config.SEED)

### Dataset

In [4]:
class KfashionDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

In [5]:
def get_test_transforms():

    return A.Compose(
        [
            A.Resize(Config.IMG_SIZE, Config.IMG_SIZE,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

### Modeling

In [6]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=Config.DEVICE)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output, nn.CrossEntropyLoss()(output,label)

class KfashionModel(nn.Module):

    def __init__(
        self,
        n_classes = Config.CLASSES,
        model_name = Config.MODEL_NAME,
        margin = Config.MARGIN,
        fc_dim = Config.FC_DIM,
        scale = Config.SCALE,
        use_fc = True,
        pretrained = True):


        super(KfashionModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif 'nfnet' in model_name:
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        if use_fc:
            self.dropout = nn.Dropout(p=0.0)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x

### Image embeddings

In [7]:
def get_image_embeddings(image_paths, model_name = Config.MODEL_NAME):
    embeds = []
    
    model = KfashionModel(model_name = model_name)
    #model = nn.DataParallel(model)
    model.eval()
    
    #if model_name == 'eca_nfnet_l0':
    #    model = replace_activations(model, torch.nn.SiLU, Mish())

    model.load_state_dict(torch.load(Config.MODEL_PATH))
    model = model.to(Config.DEVICE)
    

    image_dataset = KfashionDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=Config.BATCH_SIZE,
        pin_memory=True, 
        drop_last=False,
        num_workers=4
    )
    
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [8]:
%%time
df,image_paths = read_dataset()
df.head()

CPU times: user 551 ms, sys: 109 ms, total: 659 ms
Wall time: 2min 4s


/home/user/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,image_name,label_group,fold
0,train/street_346612_best.jpg,0,7
1,train/street_346611_best.jpg,0,5
2,train/street_346613_best.jpg,0,6
3,train/street_346621_best.jpg,0,3
4,train/street_346606_best.jpg,0,1


In [9]:
df

,image_name,label_group,fold
0,train/street_346612_best.jpg,0,7
1,train/street_346611_best.jpg,0,5
2,train/street_346613_best.jpg,0,6
3,train/street_346621_best.jpg,0,3
4,train/street_346606_best.jpg,0,1
...,...,...,...
1007688,train/sexy_164270_zipup.jpg,5061,2
1007689,train/sexy_164260_zipup.jpg,5061,7
1007690,train/sexy_164257_zipup.jpg,5061,3
1007691,train/sexy_164267_zipup.jpg,5061,6


In [10]:
image_embeddings = get_image_embeddings(image_paths)

Building Model Backbone for tf_efficientnet_b3 model


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15746/15746 [14:52<00:00, 17.64it/s]


Our image embeddings shape is (1007693, 512)


### Save darray

https://seong6496.tistory.com/142

In [11]:
np.save('traindata_embeddings.npy', image_embeddings)

In [12]:
df.image_name = '/mnt/hdd1/wearly/compatibility_rec/data/images/' + df.image_name
np.save('traindata_embeddings_path.npy',df.image_name.values)

### Load darray 

In [ ]:
np.load('traindata_embeddings.npy').shape

In [ ]:
np.load('traindata_embeddings_path.npy',allow_pickle=True).shape